## Machine Learning: Classification - Managing the quality metric of global ecological footprint  QUIZ FOR STAGE C

#### Hello Hamoye Team,

#### My name is  David Olatunji,and this is my simple solution code for Quiz for stage C. 
 
#### Regards,
#### Ibukunoluwa.

Importing the necessary libraries for the quiz.

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors
import math

Importing my local path/ directory

In [ ]:
import os, sys
os.listdir()

Loading data from my local directory

In [ ]:
data_set = pd.read_csv('Data_for_UCI_named.csv')
data_set.head()

In [ ]:
data_set.info()

In [ ]:
data_set.isna().sum()

In [ ]:
#The dataset is already clean and the dataset does not seem too imbalanced and the performance of balancing was not stated in the instructions
X = data_set.drop(columns=['stab','stabf'])
y = data_set['stabf']

In [ ]:
#split data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalised_train_df = scaler.fit_transform(x_train)
normalised_train_df = pd.DataFrame(normalised_train_df,columns=x_train.columns)
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test)
normalised_test_df = pd.DataFrame(normalised_test_df,columns=x_test.columns)

## Question 15
What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=1)
model.fit(normalised_train_df,y_train)
new_predictions = model.predict(normalised_test_df)
y_test.unique()

In [ ]:
print(classification_report(y_test,new_predictions, digits=4))

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
accuracy

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
cnf_mat = confusion_matrix(y_test, new_predictions, labels=['unstable', 'stable'])

cnf_mat

## Question 16
What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.

In [ ]:
from lightgbm import LGBMClassifier
modellgbm = LGBMClassifier(random_state=1)
modellgbm.fit(normalised_train_df, y_train)

In [ ]:
new_predictionslgbm = modellgbm.predict(normalised_test_df)
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictionslgbm)
accuracy

In [ ]:
print(classification_report(y_test,new_predictionslgbm, digits=4))

## Question 17
What is the accuracy on the test set using the xgboost classiØer? In 4 decimal places.

In [ ]:
from xgboost import XGBClassifier
modelxgb = XGBClassifier(random_state=1)
modelxgb.fit(normalised_train_df, y_train)
new_predictionsxgb = modelxgb.predict(normalised_test_df)

In [ ]:
print(classification_report(y_test,new_predictionsxgb,digits=4))

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictionsxgb)
accuracy

## Question 18
Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and
random_state = 1. What are the best hyperparameters from the randomized search CV?

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(random_state=1)
extra.fit(normalised_train_df,y_train)
new_predictionesxtra = extra.predict(normalised_test_df)

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictionesxtra)
accuracy

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,'min_samples_leaf': min_samples_leaf,'min_samples_split': min_samples_split,
                       'max_features': max_features}
utility = RandomizedSearchCV(estimator=ExtraTreesClassifier(random_state=1),
                            param_distributions=hyperparameter_grid,random_state=1)
result = utility.fit(normalised_train_df, y_train)

print('Best Score: ', result.best_score_)
print('Best Params: ', result.best_params_)

In [ ]:
#improved Extratree classifier
newex = ExtraTreesClassifier(random_state=1,n_estimators=1000,min_samples_split=2, min_samples_leaf= 8,max_features= None)
newex.fit(normalised_train_df,y_train)
new_predictionsex = newex.predict(normalised_test_df)

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictionsex)
accuracy

## Question 19
Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with
random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassiØer model
with no hyperparameter tuning?

LOWER

## Question 20
Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least
important respectively?

In [ ]:
importance = newex.feature_importances_
std = np.std([tree.feature_importances_ for tree in newex.estimators_],
             axis=0)
indices = np.argsort(importance)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(normalised_train_df.shape[1]):
    print(indices[f],"-----", normalised_train_df.columns[indices[f]],"------", importance[indices[f]])